In [ ]:
!nvidia-smi

In [ ]:
!pip install imagecodecs

In [ ]:
!pip install iterative-stratification

In [ ]:
import os

import pandas as pd
import numpy as np

from sklearn.model_selection import KFold
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

import skimage.io
import matplotlib.pylab as plt
import tensorflow as tf

import torch
import random

RANDOM_STATE = 42

import ast
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf

print(tf.__version__)

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    tf.random.set_seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

    os.environ['PYTHONHASHSEED'] = str(seed)
    os.environ['PYTHONSEED'] = str(seed)

    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)

seed_everything(RANDOM_STATE)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

We can now start loading the data. The data consists of satellite images of agricultural fields along with meta-data files.

**Update:** The data files are now updated to include only unique (Location, PlotSize) pairs and the files are named accordingly *-unique.csv

The main meta-data files are *train-unique.csv* for the training images and *test-unique.csv* for the test images.

Let's start with the training data.

In [ ]:
train = pd.read_csv('../input/lacuna2021/train-unique.csv')

There are 2 types of satellite images provided in this competition:


*   RGB color images taken at June'17, December'17, June'18, and December'18
*   Spectral band images from the Sentinel-2 satellite taken monthly over the entire year of the data point acquisition i.e. the *Year* column in the meta-data.

More details of these images can be found in the data description page. 



**DATA EXPLORATION**

We now try to do understand our data.

Let's start with the meta-data.

In [ ]:

bands = '../input/lacuna2021/ImageBands.docx'

image_size = 80

In [ ]:
print("Train meta-data shape:", train.shape)
train.head(5)

In [ ]:
train.Year.value_counts()

In [ ]:
train.Quality.value_counts()

We can see that we have data points from 2015 to 2018. We have the most data points from 2018 and the least from 2016.

The Quality column describes the confidence of the annotator in the label, with *1* being least confident and *3* being most confident.

Let's statistically summarize the remaining columns grouped by each year.

In [ ]:
train.drop('Quality', 1).groupby("Year").describe(percentiles=[])

In [ ]:
fig, ax = plt.subplots(len(train.Year.unique()), 3, figsize=(12,14))
for i, year in enumerate(train.Year.unique()):
    ax[i][0].hist(train[train.Year == year].PlotSize_acres)
    ax[i][0].set_title('PlotSize_acres')
    ax[i][1].hist(train[train.Year == year].x)
    ax[i][1].set_title('x')

    ax[i][2].hist(train[train.Year == year].y)
    ax[i][2].set_title('y')
plt.show()

Now let's visualize the images.

We need two constant value to map the (x,y) values into the pixel-space of the image.

In [ ]:
CONST_X = 10.986328125 / 2
CONST_Y = 10.985731758 / 2

We can sample a random row from the data.

We now read the RGB images for each of the 4 image dates June'17, December'17, June'18, and December'18.

In [ ]:
def read_image(path):
    image = cv2.imread(path)[:, :, ::-1]
    return image

In [ ]:
def load_gray_images(ID, load_extra=False):
    # e.g id_0b242e06 -> 0b242e06
    if load_extra:
        extra = 'extra_train-'
    else:
        extra = ''
    name = ID.split('_')[1]

    img_jun17 = read_image(f'../input/lacuna2021/{extra}planet-jun17/{extra}planet-jun17/{name}.png')
    img_dec17 = read_image(f'../input/lacuna2021/{extra}planet-dec17/{extra}planet-dec17/{name}.png')
    img_jun18 = read_image(f'../input/lacuna2021/{extra}planet-jun18/{extra}planet-jun18/{name}.png')
    img_dec18 = read_image(f'../input/lacuna2021/{extra}planet-dec18/{extra}planet-dec18/{name}.png')

    img_jun17 = cv2.resize(img_jun17, (image_size, image_size))
    img_jun18 = cv2.resize(img_jun18, (image_size, image_size))
    img_dec17 = cv2.resize(img_dec17, (image_size, image_size))
    img_dec18 = cv2.resize(img_dec18, (image_size, image_size))

    img_jun17 = cv2.cvtColor(img_jun17, cv2.COLOR_BGR2GRAY)
    img_jun18 = cv2.cvtColor(img_jun18, cv2.COLOR_BGR2GRAY)
    img_dec17 = cv2.cvtColor(img_dec17, cv2.COLOR_BGR2GRAY)
    img_dec18 = cv2.cvtColor(img_dec18, cv2.COLOR_BGR2GRAY)

    return [img_jun17 , img_dec17, img_jun18,  img_dec18]

In [ ]:
def load_RGB_images(ID, load_extra=False):
    # e.g id_0b242e06 -> 0b242e06
    if load_extra:
        extra = 'extra_train-'
    else:
        extra = ''
    name = ID.split('_')[1]

    img_jun17 = read_image(f'../input/lacuna2021/{extra}planet-jun17/{extra}planet-jun17/{name}.png')
    img_dec17 = read_image(f'../input/lacuna2021/{extra}planet-dec17/{extra}planet-dec17/{name}.png')
    img_jun18 = read_image(f'../input/lacuna2021/{extra}planet-jun18/{extra}planet-jun18/{name}.png')
    img_dec18 = read_image(f'../input/lacuna2021/{extra}planet-dec18/{extra}planet-dec18/{name}.png')

    img_jun17 = cv2.resize(img_jun17, (image_size, image_size))
    img_jun18 = cv2.resize(img_jun18, (image_size, image_size))
    img_dec17 = cv2.resize(img_dec17, (image_size, image_size))
    img_dec18 = cv2.resize(img_dec18, (image_size, image_size))

    return [img_jun17 , img_dec17, img_jun18,  img_dec18]

In [ ]:
from skimage.transform import resize

def load_Spectral_image(ID, load_extra=False, year_2015=False):
    # e.g id_0b242e06 -> 0b242e06
    if load_extra:
        extra = 'extra_train-'
    else:
        extra = ''
    
    name = ID.split('_')[1]
    if year_2015:
        root_dir = 'sentinel_for_points_collected_in_2015/sentinel_for_points_collected_in_2015'
    elif not load_extra:
        if os.path.isfile(f'../input/lacuna2021/sentinel-2-part1/sentinel/{name}.tif'):
            root_dir = 'sentinel-2-part1/sentinel'
        elif os.path.isfile(f'../input/lacuna2021/sentinel-2-part2/sentinel/{name}.tif'):
            root_dir = 'sentinel-2-part2/sentinel'
    else:
        root_dir = extra + 'sentinel' + '/extra_train-sentinel'
            
    img_sentinel = skimage.io.imread(f'../input/lacuna2021/{root_dir}/{name}.tif')
    
    img_sentinel = resize(img_sentinel, (40, 40))

    return img_sentinel

In [ ]:
def load_test_Spectral_image(ID, load_extra=False, year_2015=False):
    # e.g id_0b242e06 -> 0b242e06
    if load_extra:
        extra = 'extra_train-'
    else:
        extra = ''
    
    name = ID.split('_')[1]
    if year_2015:
        if os.path.isfile(f'../input/lacuna2021/sentinel_for_points_collected_in_2015/sentinel_for_points_collected_in_2015/sentinel_for_points_collected_in_2015/sentinel_for_points_collected_in_2015/{name}.tif'):
            root_dir = 'sentinel_for_points_collected_in_2015/sentinel_for_points_collected_in_2015'
        elif os.path.isfile(f'../input/lacuna2021/sentinel-2-part1/sentinel/{name}.tif'):
            root_dir = 'sentinel-2-part1/sentinel'
        elif os.path.isfile(f'../input/lacuna2021/sentinel-2-part2/sentinel/{name}.tif'):
            root_dir = 'sentinel-2-part2/sentinel'            
    else:
        if not load_extra:
            if os.path.isfile(f'../input/lacuna2021/sentinel-2-part1/sentinel/{name}.tif'):
                root_dir = 'sentinel-2-part1/sentinel'
            else:
                root_dir = 'sentinel-2-part2/sentinel'
                
        else:
            root_dir = extra + 'sentinel' + '/extra_train-sentinel'

    img_sentinel = skimage.io.imread(f'../input/lacuna2021/{root_dir}/{name}.tif')
    
    img_sentinel = resize(img_sentinel, (40, 40))

    return img_sentinel



Note that the image size can slightly vary e.g. 84x84 or 83x83.

It is important to account for this difference in your preprocessing code when training a model e.g. padding with zeros.

In [ ]:
train.shape

Let's inspect one of these images.

We can visualize it.

**AUXLIARY DATA**

In addition to the main training data, we also have additional data points in *auxilary_data-unique.csv*.

These points can be used to improve the training of the model.

**Update:** The number of auxilary data points has decreased after removing duplicate data points. The extra training data compensates this shortage.

In [ ]:
aux = pd.read_csv('../input/lacuna2021/auxilary_data-unique.csv')
aux.head(5)

In [ ]:
# print(get_features_fromBands(ex, bands=band_names, Band_Names = names))

In [ ]:
# train_Features = pd.DataFrame([get_features_fromBands(train['Field_ID'].values[fid_idx],bands=bands,Band_Names=names) for fid_idx in tqdm(range(len(train['Field_ID'].values))) ])
# train_Features['Field_ID'] = train['Field_ID'].values

In [ ]:
print(aux.shape)
aux.isna().sum()

In [ ]:
aux.Year.value_counts()

In [ ]:
aux.Quality.value_counts()

In [ ]:
aux.drop('Quality', 1).groupby("Year").describe(percentiles=[])

Let's also visualize a sample from the auxilary data.

In [ ]:
def compute_distance(x0, y0, x1, y1):

    a = np.array([x0, y0])
    b = np.array([x1, y1])

    dist = np.linalg.norm(b - a)

    return dist
  

**EXTRA DATA**

In addition to the main training data and auxilary data, we have annotated an extra 1000 unique data points to be used also for training in *extra_train.csv*.


In [ ]:
extra = pd.read_csv('../input/lacuna2021/extra_train.csv')
extra.head(5)

In [ ]:
extra.loc[:, 'is_extra'] = True
aux.loc[:,   'is_extra'] = False
train.loc[:, 'is_extra'] = False

In [ ]:
test = pd.read_csv('../input/lacuna2021/test.csv')
test.head(5)

In [ ]:
print(test.shape)
test.isna().sum()

In [ ]:
# test = test[test['ID'].isin(sample_submission.ID.values)]

In [ ]:
print(test.shape)

In [ ]:
test['Quality'] = 3

In [ ]:
test

In [ ]:
train_extra = pd.concat([train, aux, extra], axis=0).reset_index(drop=True)

In [ ]:
train_extra.shape

In [ ]:
df = pd.concat([train_extra,test], axis=0).reset_index(drop=True)

In [ ]:
df.shape, train.shape

In [ ]:
df.isna().sum()

In [ ]:
df

In [ ]:
df.Year.value_counts()

In [ ]:
df.Yield.fillna(value = df.groupby(['PlotSize_acres','Quality'])['Yield'].transform('mean'), inplace=True)
df.Yield.fillna(value = df.groupby(['Year'])['Yield'].transform('mean'), inplace=True)

In [ ]:
df

In [ ]:
df.Quality.value_counts()

In [ ]:
df.drop('Quality', 1).groupby("Year").describe(percentiles=[])

In [ ]:
df.isna().sum()

In [ ]:
fig, ax = plt.subplots(len(extra.Year.unique()), 3, figsize=(12,12))
for i, year in enumerate(extra.Year.unique()):
    ax[i][0].hist(extra[extra.Year == year].PlotSize_acres)
    ax[i][0].set_title('PlotSize_acres')
    ax[i][1].hist(extra[extra.Year == year].x)
    ax[i][1].set_title('x')
    ax[i][2].hist(extra[extra.Year == year].y)
    ax[i][2].set_title('y')
plt.show()

In [ ]:
train_df = df[df['ID'].isin(train_extra['ID'].values)]
test_df =  df[~df['ID'].isin(train_extra['ID'].values)]

**TEST SET**

Now let's inspect the test data in *test.csv*. We will also visualize a sample but with out the markers.

The target columns (x, y) are not provided in *test.csv*. Note that the *Quality* column is also not provided.

In [ ]:
test_df.Year.value_counts()

In [ ]:
test_df

In [ ]:
train_df

In [ ]:
# train_idx = train_df[train_df['Quality'] != 3].index
# valid_idx = train_df[train_df['Quality'] == 3].index

# train = train_df.loc[train_idx].reset_index(drop=True)
# valid = train_df.loc[valid_idx].reset_index(drop=True)

# train.shape, valid.shape

In [ ]:
predictors =  ['Yield','PlotSize_acres','Year']

In [ ]:
sentinel_feature_cols =['BANDS_FEATURES_Median_FEATURES_B7_B5_0',
 'BANDS_FEATURES_Median_FEATURES_B7_B5_1',
 'BANDS_FEATURES_Median_FEATURES_B7_B5_10',
 'BANDS_FEATURES_Median_FEATURES_B7_B5_11',
 'BANDS_FEATURES_Median_FEATURES_B7_B5_2',
 'BANDS_FEATURES_Median_FEATURES_B7_B5_3',
 'BANDS_FEATURES_Median_FEATURES_B7_B5_4',
 'BANDS_FEATURES_Median_FEATURES_B7_B5_5',
 'BANDS_FEATURES_Median_FEATURES_B7_B5_6',
 'BANDS_FEATURES_Median_FEATURES_B7_B5_7',
 'BANDS_FEATURES_Median_FEATURES_B7_B5_8',
 'BANDS_FEATURES_Median_FEATURES_B7_B5_9',
 'BANDS_FEATURES_Median_FEATURES_B7_B6_0',
 'BANDS_FEATURES_Median_FEATURES_B7_B6_1',
 'BANDS_FEATURES_Median_FEATURES_B7_B6_10',
 'BANDS_FEATURES_Median_FEATURES_B7_B6_11',
 'BANDS_FEATURES_Median_FEATURES_B7_B6_2',
 'BANDS_FEATURES_Median_FEATURES_B7_B6_3',
 'BANDS_FEATURES_Median_FEATURES_B7_B6_4',
 'BANDS_FEATURES_Median_FEATURES_B7_B6_5',
 'BANDS_FEATURES_Median_FEATURES_B7_B6_6',
 'BANDS_FEATURES_Median_FEATURES_B7_B6_7',
 'BANDS_FEATURES_Median_FEATURES_B7_B6_8',
 'BANDS_FEATURES_Median_FEATURES_B7_B6_9',
 'EVI_Max_10',
 'EVI_Max_11',
 'EVI_Max_2',
 'EVI_Max_3',
 'EVI_Max_4',
 'EVI_Max_5',
 'EVI_Max_6',
 'EVI_Max_7',
 'EVI_Max_8',
 'EVI_Max_9',
 'EVI_Median_0',
 'EVI_Median_1',
 'EVI_Median_10',
 'EVI_Median_11',
 'EVI_Median_2',
 'EVI_Median_3',
 'EVI_Median_4',
 'EVI_Median_5',
 'EVI_Median_6',
 'EVI_Median_7',
 'EVI_Median_8',
 'EVI_Median_9',
 'NDVI_Max_0',
 'NDVI_Max_10',
 'NDVI_Max_3',
 'NDVI_Max_4',
 'NDVI_Max_5',
 'NDVI_Max_6',
 'NDVI_Max_7',
 'NDVI_Max_8',
 'NDVI_Max_9',
 'NDVI_Median_0',
 'NDVI_Median_1',
 'NDVI_Median_10',
 'NDVI_Median_11',
 'NDVI_Median_2',
 'NDVI_Median_3',
 'NDVI_Median_4',
 'NDVI_Median_5',
 'NDVI_Median_6',
 'NDVI_Median_7',
 'NDVI_Median_8',
 'NDVI_Median_9',
 'NDVI_Min_0',
 'NDVI_Min_10',
 'NDVI_Min_3',
 'NDVI_Min_4',
 'NDVI_Min_5',
 'NDVI_Min_6',
 'NDVI_Min_7',
 'NDVI_Min_8',
 'NDVI_Min_9']

In [ ]:
sentinel_train_df= pd.read_csv('../input/sentinel-features/train_features.csv')
sentinel_test_df = pd.read_csv('../input/sentinel-features/test_features.csv')

In [ ]:
import gc
seed_everything(RANDOM_STATE)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False


all_images = []
train_dists = []
sentinel_features = []

# loop over the input house paths
for i,row in enumerate(train_df['ID']):
    
    row_df = train_df.loc[i]
    
    sent_row = sentinel_train_df[sentinel_train_df['ID'] == str(row)][sentinel_feature_cols].values[0]
    sentinel_features.append(sent_row)

    if row_df['is_extra']:
        images = load_RGB_images(row, load_extra=True)

        if row_df['Year'] == 2017:
            idx = random.randint(0,1)
            img = images[:2][idx]

            x0, y0 = img.shape[1]//2, img.shape[0]//2
            x1 = x0 - np.round(row_df['x'] / CONST_X * img.shape[1])
            y1 = y0 + np.round(row_df['y']/ CONST_Y * img.shape[0])

            d = compute_distance(x0 = x0, y0 = y0, x1 = x1, y1 = y1)
            
        elif row_df['Year']  == 2018:

            idx = random.randint(0,1)

            img = images[2:][idx]

            x0, y0 = img.shape[1]//2, img.shape[0]//2
            x1 = x0 - np.round(row_df['x'] / CONST_X * img.shape[1])
            y1 = y0 + np.round(row_df['y']/ CONST_Y * img.shape[0])

            d = compute_distance(x0 = x0, y0 = y0, x1 = x1, y1 = y1)
            

        else:
            idx = random.randint(0,3)

            img = images[idx]

            x0, y0 = img.shape[1]//2, img.shape[0]//2
            x1 = x0 - np.round(row_df['x'] / CONST_X * img.shape[1])
            y1 = y0 + np.round(row_df['y']/ CONST_Y * img.shape[0])

            d = compute_distance(x0 = x0, y0 = y0, x1 = x1, y1 = y1)

        train_dists.append(d)
        all_images.append(img)


    else:
        images = load_RGB_images(row, load_extra=False)

        if row_df['Year'] == 2017:

            idx = random.randint(0,1)

            img = images[:2][idx]
            x0, y0 = img.shape[1]//2, img.shape[0]//2
            x1 = x0 - np.round(row_df['x'] / CONST_X * img.shape[1])
            y1 = y0 + np.round(row_df['y']/ CONST_Y * img.shape[0])

            d = compute_distance(x0 = x0, y0 = y0, x1 = x1, y1 = y1)

                    
        elif row_df['Year'] == 2018:
            idx = random.randint(0,1)

            img = images[2:][idx]
            x0, y0 = img.shape[1]//2, img.shape[0]//2
            x1 = x0 - np.round(row_df['x'] / CONST_X * img.shape[1])
            y1 = y0 + np.round(row_df['y']/ CONST_Y * img.shape[0])

            d = compute_distance(x0 = x0, y0 = y0, x1 = x1, y1 = y1)

        else:
            idx = random.randint(0,3)

            img = images[idx]

            x0, y0 = img.shape[1]//2, img.shape[0]//2
            x1 = x0 - np.round(row_df['x'] / CONST_X * img.shape[1])
            y1 = y0 + np.round(row_df['y']/ CONST_Y * img.shape[0])

            d = compute_distance(x0 = x0, y0 = y0, x1 = x1, y1 = y1)


        train_dists.append(d)
        all_images.append(img)

        del img
        _ = gc.collect()

In [ ]:
import gc
seed_everything(RANDOM_STATE)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

X_train = np.array(all_images)
train_sentinel_features = np.array(sentinel_features)
train_ds = np.array(train_dists)

X_train.shape, train_ds.shape,train_sentinel_features.shape

In [ ]:
# Tile all images into a single image
import gc
seed_everything(RANDOM_STATE)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

all_images = []
test_sentinel_features = []

# loop over the input house paths
for i,row in enumerate(test_df['ID']):

    row_df = test_df.iloc[i]

    sent_row = sentinel_test_df[sentinel_test_df['ID'] == str(row)][sentinel_feature_cols].values[0]
    test_sentinel_features.append(sent_row)
    
    images = load_RGB_images(row, load_extra=False)

    if row_df['Year'] == 2017:

        idx = random.randint(0,1)

        img = images[:2][idx]


    elif row_df['Year'] == 2018:
        idx = random.randint(0,1)

        img = images[2:][idx]


    else:
        idx = random.randint(0,3)
        img = images[idx]
            
    all_images.append(img)

    del img
    _ = gc.collect()

In [ ]:
import gc
seed_everything(RANDOM_STATE)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

X_test = np.array(all_images)
test_sentinel_features = np.array(test_sentinel_features)

X_test.shape, test_sentinel_features.shape

In [ ]:
import gc
seed_everything(RANDOM_STATE)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

X_test_aux =  np.concatenate([test_df[predictors].values,test_sentinel_features], axis=1)
X_train_aux = np.concatenate([train_df[predictors].values,train_sentinel_features], axis=1)

In [ ]:
X_test_aux.shape, X_train_aux.shape

In [ ]:
import gc
seed_everything(RANDOM_STATE)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

ytrain = train_df[['x', 'y']].values

ytrain = np.array(ytrain.tolist()).reshape((ytrain.shape[0], 2))

ytrain.shape

In [ ]:
import keras
from keras.layers import *
from keras.optimizers import Adam
from keras.models import Model
from keras.initializers import glorot_normal
from tensorflow.keras.callbacks import LearningRateScheduler, EarlyStopping,ReduceLROnPlateau
from tensorflow.keras import backend as K
from tensorflow.keras.applications import ResNet50,ResNet152

from keras.regularizers import l2
import tensorflow as tf
import random


In [ ]:
kf = MultilabelStratifiedKFold(n_splits=20, shuffle=True, random_state=42)

In [ ]:
def build_model():

    img_inp = keras.Input(shape=(image_size, image_size,3), name="img_inputs")    
    aux_inp = keras.Input(shape=(79), name="auxillary_inputs")
    
    inp_proj = ResNet50(include_top=False, weights='imagenet')(img_inp)

    x_avg = tf.keras.layers.GlobalAveragePooling2D()(inp_proj)

    out1 = Dropout(0.5)(x_avg)

    out2 = Dense(64, activation='relu')(aux_inp)

    concat = keras.layers.Concatenate(name="concat_layer")([out1, out2])

    prediction = keras.layers.Dense(2, name="prediction")(concat)
    predict_dist =  keras.layers.Dense(1, name="predict_dist")(concat)
    
    # Model
    model = keras.Model(inputs=[img_inp, aux_inp], outputs=[prediction, predict_dist])
    
    opt = keras.optimizers.Adam(learning_rate=0.001)
    
    model.compile(
        loss= {"prediction":"huber","predict_dist": "huber"},
        loss_weights = { "prediction": 0.8,"predict_dist": 0.2 },
        optimizer=opt,
        metrics = [tf.keras.metrics.MeanAbsoluteError(name='mae')]

    )
    
    return model

In [ ]:
import gc
seed_everything(RANDOM_STATE)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

model_predictions = []

for i,(tr_index,test_index) in enumerate(kf.split(X_train, ytrain)):
  
    X_train_,y_train, Xtrain_aux = X_train[tr_index],ytrain[tr_index], X_train_aux[tr_index]

    X_valid_,y_valid ,Xvalid_aux = X_train[test_index],ytrain[test_index], X_train_aux[test_index]

    y_train_2 , y_valid_2 = train_ds[tr_index], train_ds[test_index]

    model = build_model()
    
    checkpoint_filepath = '/tmp/efnetb5_checkpoint'
    model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
                                filepath= checkpoint_filepath,
                                save_weights_only=True,
                                monitor='val_prediction_mae',
                                mode='min',
                                save_best_only=True)
    
    es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=50,
                                 mode='min',restore_best_weights=False)
    sch_cb = ReduceLROnPlateau(
        monitor='val_prediction_mae',
        factor= 0.7,
        patience= 2,
        min_lr=1e-5,
        verbose=1,
        mode='min')
        
    print()
    print(f'######### FOLD {i+1} / {kf.n_splits} ')

    model.fit([X_train_, Xtrain_aux], 
        [y_train, y_train_2],
        epochs = 300, 
        batch_size = 128,
        validation_data = ([X_valid_ , Xvalid_aux], [y_valid, y_valid_2]),
        callbacks = [es, model_checkpoint_callback, sch_cb]
        )
    model.load_weights(checkpoint_filepath)

    preds,_ = model.predict([X_test, X_test_aux])

    del model, X_train_, X_valid_, Xvalid_aux, Xtrain_aux

    model_predictions.append(preds)

In [ ]:
import gc
seed_everything(RANDOM_STATE)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

preds = np.mean(model_predictions, axis=0)

test_df[['x', 'y']] = preds

sub_df = test_df[['ID','x', 'y']]

sub_df.to_csv('sub_resnet50_final.csv', index=False)

In [ ]:
def build_resnetmodel():

    img_inp = keras.Input(shape=(image_size, image_size,3), name="img_inputs")    
    aux_inp = keras.Input(shape=(79), name="auxillary_inputs")
    
    inp_proj = ResNet152(include_top=False, weights='imagenet')(img_inp)

    x_avg = tf.keras.layers.GlobalAveragePooling2D()(inp_proj)

    out1 = Dropout(0.5)(x_avg)

    out2 = Dense(64, activation='relu')(aux_inp)
    
    concat = keras.layers.Concatenate(name="concat_layer")([out1, out2])

    prediction = keras.layers.Dense(2, name="prediction")(concat)
    predict_dist =  keras.layers.Dense(1, name="predict_dist")(concat)
    
    # Model
    model = keras.Model(inputs=[img_inp, aux_inp], outputs=[prediction, predict_dist])
    
    opt = keras.optimizers.Adam(learning_rate=0.001)
    
    model.compile(
        loss= {"prediction":"huber","predict_dist": "huber"},
        loss_weights = { "prediction": 0.8,"predict_dist": 0.2 },
        optimizer=opt,
        metrics = [tf.keras.metrics.MeanAbsoluteError(name='mae')]

    )
    
    return model

In [ ]:
import gc
seed_everything(RANDOM_STATE)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

resnetmodel_predictions = []

for i,(tr_index,test_index) in enumerate(kf.split(X_train, ytrain)):
  
    X_train_,y_train, Xtrain_aux = X_train[tr_index],ytrain[tr_index], X_train_aux[tr_index]

    X_valid_,y_valid ,Xvalid_aux = X_train[test_index],ytrain[test_index], X_train_aux[test_index]

    y_train_2 , y_valid_2 = train_ds[tr_index], train_ds[test_index]

    model = build_resnetmodel()
    
    checkpoint_filepath = '/tmp/resnet152_checkpoint'
    model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
                                filepath= checkpoint_filepath,
                                save_weights_only=True,
                                monitor='val_prediction_mae',
                                mode='min',
                                save_best_only=True)
    
    es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=50,
                                  mode='min',restore_best_weights=False)
    sch_cb = ReduceLROnPlateau(
        monitor='val_prediction_mae',
        factor= 0.7,
        patience= 2,
        min_lr=1e-5,
        verbose=1,
        mode='min')
    
    print()
    print(f'######### FOLD {i+1} / {kf.n_splits} ')

    model.fit([X_train_, Xtrain_aux], 
        [y_train, y_train_2],
        epochs = 300, 
        batch_size = 128,
        validation_data = ([X_valid_ , Xvalid_aux], [y_valid, y_valid_2]),
        callbacks = [model_checkpoint_callback, es,sch_cb]
        )
    
    model.load_weights(checkpoint_filepath)
    
    preds,_ = model.predict([X_test, X_test_aux])

    del model, X_train_, X_valid_, Xvalid_aux, Xtrain_aux

    resnetmodel_predictions.append(preds)

In [ ]:
import gc
seed_everything(RANDOM_STATE)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

respreds = np.mean(resnetmodel_predictions, axis=0)

test_df[['x', 'y']] = respreds

sub_df = test_df[['ID','x', 'y']]

sub_df.to_csv('sub_resnet152_final_20cv_folds.csv', index=False)

In [ ]:
def build_model_2():

    inp = keras.Input(shape=(76), name="inputs")
    
    x = Dense(64,  activation='relu')(inp)
    x = Dense(128, activation='relu')(x)
    x = Dense(256, activation='relu')(x)
    x = Dense(512, activation='relu')(x)
    
    out = Dropout(0.5)(x)

    prediction = keras.layers.Dense(2, name="predictions")(out)
    
    # Model
    model = keras.Model(inputs=[inp], outputs=[prediction])
    
    opt = keras.optimizers.Adam(learning_rate=0.001)
    
    model.compile(
        loss= "mae",
        optimizer=opt,
        metrics = [tf.keras.metrics.MeanAbsoluteError(name='mae')]

    )
    
    return model

In [ ]:
import gc
seed_everything(RANDOM_STATE)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
                              
X_test =   test_sentinel_features
X_train =  train_sentinel_features

print(X_test.shape,X_train.shape)

In [ ]:
gc.collect()

In [ ]:
import gc

seed_everything(RANDOM_STATE)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

nn_preds = []

for i,(tr_index,test_index) in enumerate(kf.split(X_train, ytrain)):
    
    X_train_,y_train = X_train[tr_index],ytrain[tr_index]
    X_valid_,y_valid = X_train[test_index],ytrain[test_index]

    model = build_model_2()
    
    checkpoint_filepath = '/tmp/mlp_ckpt'
    model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
                                filepath= checkpoint_filepath,
                                save_weights_only=True,
                                monitor='val_mae',
                                mode='min',
                                save_best_only=True)
    
    es = EarlyStopping(monitor='val_loss', patience=20,
                                      mode='min',restore_best_weights=False)
    sch_cb = ReduceLROnPlateau(
        monitor='val_prediction_mae',
        factor= 0.7,
        patience= 2,
        min_lr=1e-5,
        verbose=1,
        mode='min')
    print()
    print(f'######### FOLD {i+1} / {kf.n_splits} ')

    model.fit(X_train_, 
            y_train,
            epochs = 300, 
            batch_size = 64,
            validation_data = (X_valid_, y_valid),
            callbacks = [es, model_checkpoint_callback, sch_cb]
            )
    model.load_weights(checkpoint_filepath)

    preds = model.predict(X_test)

    del model, X_train_, X_valid_

    nn_preds.append(preds)

In [ ]:
import gc
seed_everything(RANDOM_STATE)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False


mlp_preds = np.mean(nn_preds, axis=0)

test_df[['x', 'y']] = mlp_preds

sub_df = test_df[['ID','x', 'y']]

sub_df.to_csv('sub_mlp_final_20cv_folds.csv', index=False)